<a href="https://colab.research.google.com/github/Abhimanyu004/CNN_projects/blob/main/fashion_img_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion = keras.datasets.fashion_mnist

In [4]:
(train_img, train_labels), (test_img, test_labels) = fashion.load_data()

In [5]:
train_img = train_img/255.0
test_img = test_img/255.0

In [6]:
train_img.shape

(60000, 28, 28)

In [7]:
train_img = train_img.reshape(len(train_img), 28,28,1)
test_img = test_img.reshape(len(test_img), 28,28,1)

In [8]:
train_img.shape

(60000, 28, 28, 1)

In [9]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('Conv_filter_1', min_value = 32, max_value = 128, step = 16),
          kernel_size = hp.Choice('convo_kernal_1', values = [3,5]),
          activation ='relu',
          input_shape = (28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_filter_2', min_value=32, max_value=64, step=16),
          kernel_size=hp.Choice('conv_kernel_2', values = [3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model

In [10]:
!pip install keras-tuner

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-11-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [12]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='output',
    project_name="mnist_fashion")

In [13]:
tuner.search(train_img, train_labels, epochs = 5, validation_split=0.1)

Trial 5 Complete [00h 01m 24s]
val_accuracy: 0.8560000061988831

Best val_accuracy So Far: 0.918666660785675
Total elapsed time: 00h 06m 18s


In [15]:
model = tuner.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 112)       2912      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        64576     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 48)                1486896   
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 1554874 (5.93 MB)
Trainable params: 1554874 (5.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
model.fit(train_img, train_labels, epochs=10, validation_split=0.1, initial_epoch=5)

Epoch 6/10
1688/1688 [==============================] - 10s 5ms/step - loss: 0.1052 - accuracy: 0.9604 - val_loss: 0.2907 - val_accuracy: 0.9097
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0830 - accuracy: 0.9690 - val_loss: 0.3424 - val_accuracy: 0.9125
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0660 - accuracy: 0.9761 - val_loss: 0.3731 - val_accuracy: 0.9007
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0506 - accuracy: 0.9809 - val_loss: 0.4398 - val_accuracy: 0.9077
Epoch 10/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0445 - accuracy: 0.9836 - val_loss: 0.4958 - val_accuracy: 0.9078
